# Wahzu Classifier

## Pre-processing

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
safe_df = pd.read_csv('safe.csv')
threat_df = pd.read_csv('threat.csv')

In [3]:
safe_df.replace(' ', np.nan, inplace=True)

In [4]:
threat_df.replace(' ', np.nan, inplace=True)

In [5]:
safe_df['Flag'] = 0
threat_df['Flag'] = 1

In [6]:
extra_columns = [
    "_source.data.win.eventdata.originalFileName",
    "_source.data.win.eventdata.image",
    "_source.data.win.eventdata.description",
    "_source.data.win.eventdata.parentCommandLine",
    "_source.data.win.eventdata.parentImage",
    "_source.data.win.eventdata.commandLine",
    "_source.data.win.eventdata.integrityLevel",
    "_source.data.win.eventdata.user",
    "_source.data.win.eventdata.targetFilename",
    "_source.data.win.eventdata.appName",
    "_source.data.win.eventdata.moduleName",
    "_source.data.win.eventdata.appPath"
]

In [7]:
safe_df[extra_columns] = np.nan

In [92]:
final_cols = [
    'agent.name',
    'data.win.system.eventID',
    'data.win.system.channel',
    'data.win.system.severityValue',
    'data.win.system.providerName',
    'rule.firedtimes',#
    'rule.level',
    'rule.groups',
    'location',
    'data.win.eventdata.logonProcessName',
    'data.win.eventdata.elevatedToken',
    'data.win.eventdata.processName',#
    'data.win.eventdata.targetDomainName',
    'data.win.eventdata.logonType',#
    'rule.mitre.technique',
    'rule.mitre.tactic',
    'syscheck.path',
    'syscheck.event',
    'syscheck.value_name',
    'syscheck.win_perm_after',
    'data.win.eventdata.p1',
    'data.win.eventdata.serviceType',#
    'data.vulnerability.severity',#
    'data.vulnerability.cve',#
    'data.vulnerability.cvss.cvss3.base_score',# scale
    'data.win.eventdata.originalFileName',
    'data.win.eventdata.image',
    #'data.win.eventdata.integrityLevel',
    'Flag'
]

print(len(final_cols))

28


In [93]:
safe_df.columns = safe_df.columns.str.replace('^_source.', '', regex=True)
threat_df.columns = threat_df.columns.str.replace('^_source.', '', regex=True)

In [94]:
safe_final_df = safe_df[final_cols]
threat_final_df = threat_df[final_cols]

In [95]:
combined_df = pd.concat([safe_final_df, threat_final_df], axis=0)

In [96]:
filtering_df = combined_df

In [99]:
# Duplicate dataframe for filtering with blacklist later
combined_df = combined_df.drop(['data.win.eventdata.serviceType','data.vulnerability.severity','data.vulnerability.cve'], axis=1)

In [98]:
combined_df.head()

,agent.name,data.win.system.eventID,data.win.system.channel,data.win.system.severityValue,data.win.system.providerName,rule.firedtimes,rule.level,rule.groups,location,data.win.eventdata.logonProcessName,data.win.eventdata.elevatedToken,data.win.eventdata.processName,data.win.eventdata.targetDomainName,data.win.eventdata.logonType,rule.mitre.technique,rule.mitre.tactic,syscheck.path,syscheck.event,syscheck.value_name,syscheck.win_perm_after,data.win.eventdata.p1,data.win.eventdata.serviceType,data.vulnerability.severity,data.vulnerability.cve,data.vulnerability.cvss.cvss3.base_score,data.win.eventdata.originalFileName,data.win.eventdata.image,Flag
0,Windows,1014,System,WARNING,Microsoft-Windows-DNS-Client,1,5,"[""windows"",""windows_system""]",EventChannel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Windows,4624,Security,AUDIT_SUCCESS,Microsoft-Windows-Security-Auditing,6,3,"[""windows"",""windows_security"",""authentication_...",EventChannel,Advapi,%%1842,C:\\Windows\\System32\\services.exe,NT AUTHORITY,5,"[""Valid Accounts""]","[""Defense Evasion"",""Persistence"",""Privilege Es...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,ubuntu,NaN,NaN,NaN,NaN,1,7,"[""syslog"",""dpkg"",""config_changed""]",/var/log/dpkg.log,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,ubuntu,NaN,NaN,NaN,NaN,4,7,"[""sca""]",sca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,ubuntu,NaN,NaN,NaN,NaN,16,7,"[""sca""]",sca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [13]:
# Replace NaN values with 'unmodified' in the 'syscheck.event' column
combined_df['syscheck.event'] = combined_df['syscheck.event'].fillna('unmodified')

In [14]:
combined_df['data.win.eventdata.image'] = combined_df['data.win.eventdata.image'].apply(lambda x: 1 if isinstance(x, str) and x.startswith('C:\\Windows\\System32\\') else 0)

In [15]:
# Columns to One-Hot Encode
one_hot_columns = ['agent.name', 'data.win.system.eventID', 'data.win.system.channel', 'location', 'data.win.system.providerName', 'rule.mitre.technique', 'rule.mitre.tactic', 'syscheck.event']

# Columns to Log Transform
log_transform_columns = ['rule.firedtimes']

# Columns to Label Encode
# Structured dictionary to store column names and their respective orderings
label_encoding_column_order_mapping = {
    'data.win.system.severityValue': [np.nan, 'INFORMATION', 'AUDIT_SUCCESS', 'WARNING', 'ERROR', 'AUDIT_FAILURE'],
}

# Dictionary with acceptable values for specific columns
acceptable_values = {
    'data.win.eventdata.targetDomainName': ['NT AUTHORITY'],  # Only 'NT AUTHORITY' is acceptable for column 'data.win.eventdata.targetDomainName'
    'data.win.eventdata.originalFileName': ['wannacry.exe', 'notpetya.exe', 'trickbot.exe', 'emotet.exe', 'ryuk.exe', 'locky.exe', 'cryptolocker.exe', 'keylogger.exe', 'winspy.exe', 'darkcomet.exe', 'nanocore.exe', 'teamviewer.exe', 'anydesk.exe', 'radmin.exe', 'vncserver.exe', 'remcmd.exe', 'mimikatz.exe', 'procdump.exe', 'dumpert.exe', 'pwdump.exe', 'nmap.exe', 'angryipscanner.exe', 'metasploit.exe', 'sqlmap.exe', 'xmrig.exe', 'minerd.exe', 'cryptonight.exe', 'ccminer.exe', 'svchost.exe', 'explorer.exe', 'lsass.exe', 'csrss.exe', 'winlogon.exe', 'wscript.exe', 'cscript.exe', 'powershell.exe', 'mshta.exe', 'regsvr32.exe', 'installutil.exe', 'msiexec.exe', 'schtasks.exe', 'certutil.exe', 'bitsadmin.exe', 'ftp.exe', 'sc.exe', 'driverquery.exe', 'rundll32.exe', 'taskeng.exe', 'conhost.exe'] ,
    'data.win.eventdata.logonType': ['wannacry.exe', 'notpetya.exe'], # New list required
    'data.win.eventdata.processName': ['wannacry.exe', 'notpetya.exe'] # New list required
    
}

# List of columns for missing value indicator
missing_value_columns = ['data.win.eventdata.logonProcessName ', 'data.win.eventdata.elevatedToken', 'syscheck.path', 'syscheck.value_name', 'syscheck.win_perm_after', 'data.win.eventdata.p1']

# List of columns for scaling
scaling_columns = []

In [16]:
combined_df['data.win.system.severityValue'].unique()

array(['WARNING', 'AUDIT_SUCCESS', nan, 'INFORMATION', 'ERROR',
       'AUDIT_FAILURE'], dtype=object)

In [17]:
from sklearn.preprocessing import OneHotEncoder

for column in one_hot_columns:
    encoder = OneHotEncoder(sparse_output=False, drop='first')  # Drop first to avoid dummy variable trap
    encoded = encoder.fit_transform(combined_df[[column]])
    encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out([column]))

    # Reset the index to avoid concatenation issues
    combined_df = combined_df.drop(column, axis=1).reset_index(drop=True)
    encoded_df = encoded_df.reset_index(drop=True)

    # Concatenate the two DataFrames
    combined_df = pd.concat([combined_df, encoded_df], axis=1)

In [18]:
# Log Transformation
for column in log_transform_columns:
    # Check for non-positive values to avoid issues with log(0) or log of negative numbers
    if (combined_df[column] <= 0).any():
        raise ValueError(f"Column '{column}' contains non-positive values, cannot apply log transform.")
    combined_df[column] = np.log(combined_df[column])

In [19]:
# Function to map each column based on its order
def label_encode_with_custom_order(df, label_encoding_column_order_mapping):
    for column, order in label_encoding_column_order_mapping.items():
        # Create a dictionary mapping the custom order to numerical values
        order_mapping = {value: idx for idx, value in enumerate(order)}
        
        # Apply the mapping to the column
        df[column] = df[column].map(order_mapping)
    
    return df

# Apply the custom encoding
combined_df = label_encode_with_custom_order(combined_df, label_encoding_column_order_mapping)

In [20]:
# Acceptable Values Mapping
for column, acceptable in acceptable_values.items():
    if column in combined_df.columns:
        combined_df[column] = combined_df[column].apply(lambda x: 1 if x in acceptable else 0)

In [21]:
# Missing Value Indicator
for column in missing_value_columns:
    if column in combined_df.columns:
        combined_df[column] = combined_df[column].apply(lambda x: 0 if pd.isnull(x) else 1)

In [22]:
# Scale data.vulnerability.cvss.cvss3.base_score to get between 0-1
combined_df['data.vulnerability.cvss.cvss3.base_score'] = combined_df['data.vulnerability.cvss.cvss3.base_score'].fillna(0)  # Replace NaN with 0
combined_df['data.vulnerability.cvss.cvss3.base_score'] = combined_df['data.vulnerability.cvss.cvss3.base_score'].astype(float)  # Convert to float for division
combined_df['data.vulnerability.cvss.cvss3.base_score'] = combined_df['data.vulnerability.cvss.cvss3.base_score'].apply(lambda x: x / 10)  # Scale the values

In [23]:
print("Processed DataFrame:")
display(combined_df)

Processed DataFrame:


,data.win.system.severityValue,rule.firedtimes,rule.level,rule.groups,data.win.eventdata.logonProcessName,data.win.eventdata.elevatedToken,data.win.eventdata.processName,data.win.eventdata.targetDomainName,data.win.eventdata.logonType,syscheck.path,...,"rule.mitre.tactic_[""Lateral Movement""]","rule.mitre.tactic_[""Persistence"",""Privilege Escalation""]","rule.mitre.tactic_[""Persistence""]","rule.mitre.tactic_[""Privilege Escalation"",""Defense Evasion"",""Execution""]","rule.mitre.tactic_[""Privilege Escalation"",""Defense Evasion""]","rule.mitre.tactic_[""Privilege Escalation"",""Persistence""]",rule.mitre.tactic_nan,syscheck.event_deleted,syscheck.event_modified,syscheck.event_unmodified
0,3,0.000000,5,"[""windows"",""windows_system""]",NaN,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,2,1.791759,3,"[""windows"",""windows_security"",""authentication_...",Advapi,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,0.000000,7,"[""syslog"",""dpkg"",""config_changed""]",NaN,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0,1.386294,7,"[""sca""]",NaN,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0,2.772589,7,"[""sca""]",NaN,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5133,1,0.000000,3,"[""sysmon"",""sysmon_eid1_detections"",""windows""]",NaN,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5134,1,0.693147,4,"[""sysmon"",""sysmon_eid1_detections"",""windows""]",NaN,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5135,1,2.197225,3,"[""sysmon"",""sysmon_eid1_detections"",""windows""]",NaN,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5136,1,2.197225,4,"[""sysmon"",""sysmon_eid1_detections"",""windows""]",NaN,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [24]:
# Convert 'rule.level' to numeric, forcing invalid parsing to NaN
combined_df['rule.level'] = pd.to_numeric(combined_df['rule.level'], errors='coerce')

# Define the bins and labels
bins = [0, 4, 10, 15]
labels = ['LOW', 'MED', 'HIGH']

# Binning the 'rule.level' column
combined_df['rule.level'] = pd.cut(
    combined_df['rule.level'],
    bins=bins,
    labels=labels,
    include_lowest=True,
    right=True
)

# Handle any NaN values that may result from out-of-bounds values
combined_df['rule.level'] = combined_df['rule.level'].fillna('LOW')

# Map the binned categories to specific numerical values
category_mapping = {'LOW': 0, 'MED': 1, 'HIGH': 2}
combined_df['rule.level'] = combined_df['rule.level'].map(category_mapping)

In [25]:
import ast

# Step 1: Parse the 'rule.groups' column from string representation of lists to actual lists
combined_df['rule.groups'] = combined_df['rule.groups'].apply(ast.literal_eval)

# Step 2: Extract all unique individual values from the 'rule.groups' column
unique_values = set()
for groups in combined_df['rule.groups']:
    unique_values.update(groups)

unique_values = sorted(unique_values)  # Optional: sort the unique values for consistency

# Step 3: Create a new column for each unique value and encode it
for value in unique_values:
    combined_df[value] = combined_df['rule.groups'].apply(lambda x: 1 if value in x else 0)

# Step 4: Remove the original 'rule.groups' column
combined_df = combined_df.drop('rule.groups', axis=1)

In [26]:
combined_df.head()

,data.win.system.severityValue,rule.firedtimes,rule.level,data.win.eventdata.logonProcessName,data.win.eventdata.elevatedToken,data.win.eventdata.processName,data.win.eventdata.targetDomainName,data.win.eventdata.logonType,syscheck.path,syscheck.value_name,...,sysmon_process-anomalies,system_error,systemd,time_changed,usb,vulnerability-detector,windows,windows_application,windows_security,windows_system
0,3,0.000000,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
1,2,1.791759,0,Advapi,1,0,1,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,0,0.000000,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1.386294,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,2.772589,1,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
combined_df.shape[1]

189

## Neural Network Training

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Assuming the preprocessed data is in a DataFrame 'data'
data = combined_df

# Separate features and target
X = data.drop(columns=['Flag'])
y = data['Flag']

In [43]:
X.shape[1]

188

In [44]:
combined_df.shape[1]

189

In [45]:
# Convert categorical features to one-hot encoded variables
X = pd.get_dummies(X)

In [46]:
X.shape[1]

192

In [47]:
# One-hot encode the target variable if it's a categorical classification problem
y = to_categorical(y)

# Split into training and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
X_train.shape[1]

192

In [ ]:
# Standardize the features (mean=0, variance=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # First hidden layer
model.add(Dense(32, activation='relu'))  # Second hidden layer
model.add(Dense(y_train.shape[1], activation='softmax'))  # Output layer with softmax for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f'Test accuracy: {test_accuracy}')

Epoch 1/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7974 - loss: 0.4322 - val_accuracy: 0.8844 - val_loss: 0.1988
Epoch 2/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 478us/step - accuracy: 0.8727 - loss: 0.1995 - val_accuracy: 0.8735 - val_loss: 0.2031
Epoch 3/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - accuracy: 0.8735 - loss: 0.1950 - val_accuracy: 0.8820 - val_loss: 0.1915
Epoch 4/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 470us/step - accuracy: 0.8655 - loss: 0.1931 - val_accuracy: 0.8747 - val_loss: 0.1983
Epoch 5/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 480us/step - accuracy: 0.8785 - loss: 0.1927 - val_accuracy: 0.8893 - val_loss: 0.1938
Epoch 6/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 469us/step - accuracy: 0.8716 - loss: 0.1919 - val_accuracy: 0.8820 - val_loss: 0.1921
Epoch 7/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 468us/step - accuracy: 0.8770 - loss: 0.1842 - val_accuracy: 0.8808 - val_loss: 0.1933
Epoch 8/50
103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step - accuracy: 0.8743 - loss: 0.1927 - va

In [82]:
# Set pandas to display all columns without truncation
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent breaking into new lines

In [83]:
test_df = X_train.sample(n=10, random_state=None)

In [84]:
test_df.head(10)

,data.win.system.severityValue,rule.firedtimes,data.win.eventdata.elevatedToken,data.win.eventdata.processName,data.win.eventdata.targetDomainName,data.win.eventdata.logonType,syscheck.path,syscheck.value_name,syscheck.win_perm_after,data.win.eventdata.p1,data.vulnerability.cvss.cvss3.base_score,data.win.eventdata.originalFileName,data.win.eventdata.image,agent.name_ubuntu,data.win.system.eventID_1000,data.win.system.eventID_10000,data.win.system.eventID_10001,data.win.system.eventID_1001,data.win.system.eventID_10010,data.win.system.eventID_1003,data.win.system.eventID_1014,data.win.system.eventID_102,data.win.system.eventID_1026,data.win.system.eventID_1033,data.win.system.eventID_1035,data.win.system.eventID_1040,data.win.system.eventID_11,data.win.system.eventID_11707,data.win.system.eventID_12289,data.win.system.eventID_13,data.win.system.eventID_16384,data.win.system.eventID_17,data.win.system.eventID_1796,data.win.system.eventID_20,data.win.system.eventID_3,data.win.system.eventID_300,data.win.system.eventID_301,data.win.system.eventID_302,data.win.system.eventID_326,data.win.system.eventID_4004,data.win.system.eventID_4616,data.win.system.eventID_4624,data.win.system.eventID_4625,data.win.system.eventID_4634,data.win.system.eventID_4647,data.win.system.eventID_6000,data.win.system.eventID_7040,data.win.system.eventID_7045,data.win.system.eventID_8224,data.win.system.eventID_86,data.win.system.eventID_nan,data.win.system.channel_Microsoft-Windows-Sysmon/Operational,data.win.system.channel_Security,data.win.system.channel_System,data.win.system.channel_nan,location_/var/log/dpkg.log,location_/var/log/kern.log,location_/var/log/syslog,location_EventChannel,location_active-response\active-responses.log,location_netstat listening ports,location_sca,location_syscheck,location_vulnerability-detector,location_wazuh-agent,location_wazuh-monitord,location_wazuh-remoted,data.win.system.providerName_Application Error,data.win.system.providerName_BTHUSB,data.win.system.providerName_ESENT,data.win.system.providerName_Microsoft-Windows-CertificateServicesClient-CertEnroll,data.win.system.providerName_Microsoft-Windows-DNS-Client,data.win.system.providerName_Microsoft-Windows-DistributedCOM,data.win.system.providerName_Microsoft-Windows-Search,data.win.system.providerName_Microsoft-Windows-Security-Auditing,data.win.system.providerName_Microsoft-Windows-Security-SPP,data.win.system.providerName_Microsoft-Windows-Sysmon,data.win.system.providerName_Microsoft-Windows-TPM-WMI,data.win.system.providerName_Microsoft-Windows-WindowsUpdateClient,data.win.system.providerName_Microsoft-Windows-Winlogon,data.win.system.providerName_MsiInstaller,data.win.system.providerName_Service Control Manager,data.win.system.providerName_VSS,data.win.system.providerName_Windows Error Reporting,data.win.system.providerName_nan,"rule.mitre.technique_[""Account Discovery"",""PowerShell""]","rule.mitre.technique_[""Account Discovery"",""Windows Command Shell""]","rule.mitre.technique_[""Account Discovery""]","rule.mitre.technique_[""Application Shimming""]","rule.mitre.technique_[""Command and Scripting Interpreter""]","rule.mitre.technique_[""Create Account""]","rule.mitre.technique_[""Deobfuscate/Decode Files or Information""]","rule.mitre.technique_[""Disable or Modify System Firewall""]","rule.mitre.technique_[""Disable or Modify Tools""]","rule.mitre.technique_[""File Deletion"",""Data Destruction"",""Modify Registry""]","rule.mitre.technique_[""File Deletion""]","rule.mitre.technique_[""Ingress Tool Transfer"",""Command and Scripting Interpreter""]","rule.mitre.technique_[""Ingress Tool Transfer""]","rule.mitre.technique_[""Lateral Tool Transfer""]","rule.mitre.technique_[""Local Account""]","rule.mitre.technique_[""Modify Registry""]","rule.mitre.technique_[""Network Share Discovery"",""Windows Command Shell""]","rule.mitre.technique_[""Network Share Discovery""]","rule.mitre.technique_[""Non-Application Layer Protocol""]","rule.mitre.technique_[""Obfusc

In [85]:
test_df.shape[1]

192

In [86]:
# Predict the class probabilities
predictions = model.predict(test_df)

# Convert predicted probabilities to class labels (e.g., class 0, class 1, etc.)
predicted_classes = np.argmax(predictions, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [132]:
# Create an empty list to store flagged logs
flagged_logs = []

In [133]:
# Iterate over predicted classes and original logs
for i, pred_class in enumerate(predicted_classes):
    if pred_class == 1:  # If the predicted class is 1 (Flag = 1)
        flagged_logs.append(test_df.iloc[i].to_dict())  # Add the log to the flagged_logs array

# flagged_logs now contains logs where the predicted class is 1

In [134]:
print(len(flagged_logs))

5


In [135]:
print(flagged_logs)

[{'data.win.system.severityValue': 1, 'rule.firedtimes': 5.5254529391317835, 'data.win.eventdata.elevatedToken': 0, 'data.win.eventdata.processName': 0, 'data.win.eventdata.targetDomainName': 0, 'data.win.eventdata.logonType': 0, 'syscheck.path': 0, 'syscheck.value_name': 0, 'syscheck.win_perm_after': 0, 'data.win.eventdata.p1': 0, 'data.vulnerability.cvss.cvss3.base_score': 0.0, 'data.win.eventdata.originalFileName': 0, 'data.win.eventdata.image': 0, 'agent.name_ubuntu': 0.0, 'data.win.system.eventID_1000': 0.0, 'data.win.system.eventID_10000': 0.0, 'data.win.system.eventID_10001': 0.0, 'data.win.system.eventID_1001': 0.0, 'data.win.system.eventID_10010': 0.0, 'data.win.system.eventID_1003': 0.0, 'data.win.system.eventID_1014': 0.0, 'data.win.system.eventID_102': 0.0, 'data.win.system.eventID_1026': 0.0, 'data.win.system.eventID_1033': 0.0, 'data.win.system.eventID_1035': 0.0, 'data.win.system.eventID_1040': 0.0, 'data.win.system.eventID_11': 1.0, 'data.win.system.eventID_11707': 0.0,

In [136]:
filtering_df.head()

,agent.name,data.win.system.eventID,data.win.system.channel,data.win.system.severityValue,data.win.system.providerName,rule.firedtimes,rule.level,rule.groups,location,data.win.eventdata.logonProcessName,data.win.eventdata.elevatedToken,data.win.eventdata.processName,data.win.eventdata.targetDomainName,data.win.eventdata.logonType,rule.mitre.technique,rule.mitre.tactic,syscheck.path,syscheck.event,syscheck.value_name,syscheck.win_perm_after,data.win.eventdata.p1,data.win.eventdata.serviceType,data.vulnerability.severity,data.vulnerability.cve,data.vulnerability.cvss.cvss3.base_score,data.win.eventdata.originalFileName,data.win.eventdata.image,Flag
0,Windows,1014,System,WARNING,Microsoft-Windows-DNS-Client,1,5,"[""windows"",""windows_system""]",EventChannel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Windows,4624,Security,AUDIT_SUCCESS,Microsoft-Windows-Security-Auditing,6,3,"[""windows"",""windows_security"",""authentication_...",EventChannel,Advapi,%%1842,C:\\Windows\\System32\\services.exe,NT AUTHORITY,5,"[""Valid Accounts""]","[""Defense Evasion"",""Persistence"",""Privilege Es...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,ubuntu,NaN,NaN,NaN,NaN,1,7,"[""syslog"",""dpkg"",""config_changed""]",/var/log/dpkg.log,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,ubuntu,NaN,NaN,NaN,NaN,4,7,"[""sca""]",sca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,ubuntu,NaN,NaN,NaN,NaN,16,7,"[""sca""]",sca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [137]:
further_flagged_logs = filtering_df[filtering_df['data.win.eventdata.serviceType'].notnull()].apply(lambda row: row.to_dict(), axis=1).tolist()

In [138]:
len(further_flagged_logs)

9

In [114]:
flagged_logs = filtering_df[filtering_df['data.vulnerability.cve'].notnull()].apply(lambda row: row.to_dict(), axis=1).tolist()

## LLM

In [116]:
import requests
import json

In [118]:
url = "http://localhost:11434/api/generate"

In [119]:
headers = {
    "Content-Type": "application/json"
}

In [147]:
special_prompt = """You are a cybersecurity analyst specializing in incident response. You are tasked with analyzing a set of processed system logs for potential malicious activity. Based on these logs, generate a detailed Incident Response Report. Identify any suspicious or malicious events in the logs, explain their significance, and provide actionable recommendations for containment, remediation, and prevention of further attacks. Be sure to focus on any potential signs of command and control (C2) activity, tool transfers, file tampering, or privilege escalation.

Include in your report:

    Summary of the incident.
    Key suspicious activities and indicators (e.g., specific Event IDs, MITRE ATT&CK tactics, elevated privileges).
    Details on affected systems or devices involved.
    Clear recommendations for next steps, such as containment, remediation, and future mitigation.
    Conclusion on the potential impact and any further investigation needed.

Here are the logs for analysis:""" + str(flagged_logs)
print(special_prompt)

You are a cybersecurity analyst specializing in incident response. You are tasked with analyzing a set of processed system logs for potential malicious activity. Based on these logs, generate a detailed Incident Response Report. Identify any suspicious or malicious events in the logs, explain their significance, and provide actionable recommendations for containment, remediation, and prevention of further attacks. Be sure to focus on any potential signs of command and control (C2) activity, tool transfers, file tampering, or privilege escalation.

Include in your report:

    Summary of the incident.
    Key suspicious activities and indicators (e.g., specific Event IDs, MITRE ATT&CK tactics, elevated privileges).
    Details on affected systems or devices involved.
    Clear recommendations for next steps, such as containment, remediation, and future mitigation.
    Conclusion on the potential impact and any further investigation needed.

Here are the logs for analysis:[{'data.win.sys

In [148]:
data = {
    "model": "llama3.2",
    "prompt": special_prompt,
    "stream": False
}

In [149]:
response = requests.post(url, headers=headers, data=json.dumps(data))

In [150]:
if response.status_code == 200:
    response_text = response.text
    data = json.loads(response_text)
    actual_response = data["response"]
    print(actual_response)
else:
    print( "Error:", response.status_code, response.text)

This appears to be a JSON object containing various rules and events from a security monitoring system. Here's a breakdown of what each part represents:

**Security Rules:**

* `rule.mitre.technique_*`: These are specific techniques used in attacks, such as "Windows Command Shell" or "Visual Basic".
* `rule.mitre.tactic_*`: These are high-level tactics used by attackers, such as "Command and Control", "Credential Access", etc.
* `rule.level_0`, `rule.level_1`, `rule.level_2`: These represent the severity levels of each rule, with level 0 being the lowest (false) and level 2 being the highest (true).
* The other rules seem to be specific events or conditions that are checked during monitoring.

**Events:**

* `syscheck.event_deleted`, `syscheck.event_modified`, `syscheck.event_unmodified`: These indicate whether a system check event has been deleted, modified, or unchanged.
* `authentification_success` and `authentication_failed`: These indicate successful and failed authentication even